### Analisi di testi con GloVe e Word2vec

Inizieremo da GloVe che può essere scaricato dal relativo repository GitHub. Useremo una versione piccola e anche non molto aggiornata degli embedding per motivi di spazio.

```bash
$ curl -o <Cartella di destinazione>/glove.6B.zip https://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip
$ cd <Cartella di destinazione>
$ unzip glove.6B.zip
```

Questo file contiene un archivio di circa 822MB e va estratto dall'archivio ottenendo il file ```glove.6B.300d.txt``` che ha una dimensione di oltre 1GB. 

Lavorando su Google Colab, il file va uploadato su Drive e il Drive va montato nel runtime. 

```python
from google.colab import drive

drive.mount('/content/gdrive')
```

Se non si dispone di spazio sufficiente sul proprio Drive allora è opportuno che, una volta creato il notebook e connesso il relativo runtime, si apra l'icona del filesytem e, all'interno della cartella ```/content``` si faccia il drag & drop del file stesso in modo da usare l'ampio spazio fornito direttamente dal runtime. 

Ovviamente alla fine della sessione si perderà tutto il contenuto e si dovrà ripetere il processo in caso che il file debba essere usato nuovamente.

In [ ]:
import csv
import nltk
import numpy as np

In [ ]:
root = "/home/rpirrone/src/"     # Da personalizzare con il proprio percorso che contiene gli embedding


def load_glove(glove_path):

    print("Loading glove vectors ...")
    with open(glove_path, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        glove_embeddings = {line[0]: np.array(list(map(float, line[1:])))
                for line in reader}
    print("Glove vectors loaded")
    return glove_embeddings

glove_embeddings = load_glove(root+"glove.6B.300d.txt")

In [ ]:
# Analizziamo un attimo il corpus
print(f"{len(glove_embeddings)} vettori da {glove_embeddings['cat'].shape[0]} elementi ciascuno\n")

In [ ]:
## To find the nearest neighbors of a word
def find_nearest(word, glove_embeddings, k=5):
  
  distances = []
  word_vec = glove_embeddings[word]
  
  for w, vec in glove_embeddings.items():
    distance = np.linalg.norm(word_vec - vec)
    distances.append((w, distance))
  distances = sorted(distances, key=lambda x: x[1])
  
  return distances[:k]

print(find_nearest('cat', glove_embeddings))
print(find_nearest('water', glove_embeddings))

In [ ]:
## Calcoliamo le analogie tra parole con la regola del parallelogramma
def find_analogy(a, b, c, glove_embeddings):
  a_vec = glove_embeddings[a]
  b_vec = glove_embeddings[b]
  c_vec = glove_embeddings[c]
  d_vec = b_vec - a_vec + c_vec
  distances = []
  
  for w, vec in glove_embeddings.items():
    distance = np.linalg.norm(d_vec - vec)  
    distances.append((w, distance))
  distances = sorted(distances, key=lambda x: x[1])
  
  return distances[:1][0] # restituiamo direttamente la coppia (parola, distanza dal valore ideale)

word, distance = find_analogy('king', 'man', 'queen', glove_embeddings)
print(f"'king' : 'man' --> 'queen' : '{word}'. Distanza: {distance}\n\n")

word, distance = find_analogy('paris', 'france', 'rome', glove_embeddings)
print(f"'paris' : 'france' --> 'rome' : '{word}'. Distanza: {distance}\n\n")

word, distance = find_analogy('woman', 'actress', 'man', glove_embeddings)
print(f"'woman' : 'actress' --> 'man' : '{word}'. Distanza: {distance}\n\n")

word, distance = find_analogy('pianist', 'piano', 'guitarist', glove_embeddings)
print(f"'pianist' : 'piano' --> 'guitarist' : '{word}'. Distanza: {distance}\n\n")


Visualizziamo gli embedding in uno spazio bidimensionale usando l'algoritmo t-SNE per la riduzione della dimensionalità. t-SNE converte le somiglianze tra punti dati in probabilità congiunte e cerca di minimizzare la divergenza di Kullback-Leibler tra le probabilità congiunte dell’embedding a bassa dimensionalità e quelle dei dati ad alta dimensionalità.

La funzione di costo di t-SNE non è convessa, cioè con diverse inizializzazioni si possono ottenere risultati differenti.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

words = ['woman', 'actress', 'man', 'actor', 'pianist',
         'piano', 'guitarist', 'guitar', 'king', 'queen']

def visualize_embeddings(embeddings, words):
    
    pca = PCA(n_components=len(words),svd_solver='full')
    tsne = TSNE(n_components=2, random_state=0, perplexity=len(words)-1)
    
    embedding_vectors = np.array([embeddings[word] for word in words])
    pca_embeddings = pca.fit_transform(embedding_vectors)
    two_d_embeddings = tsne.fit_transform(pca_embeddings)

    plt.figure(figsize=(8, 8))
    for i, word in enumerate(words):
        x, y = two_d_embeddings[i, :]
        plt.scatter(x, y)
        plt.annotate(word, (x, y), xytext=(5, 2),
                     textcoords="offset points", ha="right", va="bottom")
    plt.show()

glove_words = [word for word in words if word in glove_embeddings]
visualize_embeddings(glove_embeddings, glove_words)

In [ ]:
from nltk.corpus import brown
from pprint import pprint

# Scarichiamo il Brown corpus usando NLTK

try:
    nltk.data.find('corpora/brown')
except LookupError:
    nltk.download('brown')

print("="*50)
print("ANALISI DEL CORPUS BROWN")
print("="*50)

categories = brown.categories()
words_brown = brown.words(categories=categories)
sentences = brown.sents()

print(f"{len(categories)} categorie:\n\n")
pprint(categories)

print(f"\n\n{len(sentences)} frasi:\n\nEsempi:\n")
for sentence in sentences[:2]:
    print(sentence)

print(f"{len(words_brown)} parole.\n\n")

print("="*50)

In [ ]:
glove_words = []
for sent in brown.sents()[:4]:
    glove_words.extend([word for word in sent if word in glove_embeddings])
visualize_embeddings(glove_embeddings, glove_words)

Wrd2vec lo prendiamo da Gensim, https://radimrehurek.com/gensim/index.html, che è una libreria open source Python per addestrare e gestire word embedding differenti.

In [ ]:
import gensim

# Creiamo un training set e addestriamo il modello Word2vec della libreria Gensim
train_set = brown.sents()[:10000]
model = gensim.models.Word2Vec(train_set)

In [ ]:
# Salviamo e carichiamo il modello
model.save('brown.embedding')
new_model = gensim.models.Word2Vec.load('brown.embedding')

In [ ]:
# Stampiamo le caratteristiche del modello appreso
print(f"{len(new_model.wv)} embedding da {len(new_model.wv['university'])} elementi ciascuno.\n")

In [ ]:
# Calcolo della 5 parole più vicine alla lista di parole già usata per GloVe
words = ['university', 'school', 'jury', 'investigation', 'movie']
for word in words:
    print(f"\n\nLe cinque parole più simili a '{word}' sono:\n")
    pprint(new_model.wv.most_similar(positive=[word],topn=5))


In [ ]:
# Visualizziamo gli embedding Word2vec
    
w2v_words = [word for word in words if word in new_model.wv]
visualize_embeddings(new_model.wv, w2v_words)

Il modello che abbiamo usato è addestrato su pochi termini. Carichiamone uno pre-addestrato per effettuare il ragionamento per analogia. 

In [ ]:
try:
    nltk.data.find('models/word2vec_sample')
except LookupError:
    nltk.download('word2vec_sample')

word2vec_sample = str(nltk.data.find('models/word2vec_sample/pruned.word2vec.txt')) # otteniamo il path

# carichiamo il modello preaddestrato che è in un formato diverso
# per cui si accede direttamente dal modello senza la proprietà 'wv'
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

In [ ]:

# Ragionamento per analogia
word, similarita = model.most_similar(positive=['woman','king'], negative=['man'], topn = 1)[0]

print(f"'woman' : 'king' --> 'man' : '{word}'. Similarità: {similarita}\n\n")

word, similarita = model.most_similar(positive=['Paris','Germany'], negative=['Berlin'], topn = 1)[0]
print(f"'Paris' : 'Germany' --> 'Berlin' : '{word}'. Distanza: {similarita}\n\n")

word, similarita = model.most_similar(positive=['woman','actor'], negative=['man'], topn = 1)[0]
print(f"'woman' : 'actor' --> 'man' : '{word}'. Distanza: {similarita}\n\n")

word, similarita = model.most_similar(positive=['guitarist','piano'], negative=['pianist'], topn = 1)[0]
print(f"'guitarist' : 'piano' --> 'pianist' : '{word}'. Distanza: {similarita}\n\n")

In [ ]:
# Visualizziamo gli embedding Word2vec

words = ['man', 'woman', 'king', 'queen', 'Pairs', 'Berlin', 'France', 'Germany', 'actor', 
         'actress', 'pianist', 'guitarist', 'piano', 'guitar']
    
w2v_words = [word for word in words if word in model]
visualize_embeddings(model, w2v_words)